<a href="https://colab.research.google.com/github/IS5882/UU-TAX/blob/main/Sub-task%201%20Code/UU_Tax_Subtask1_French.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0] Imports and reading all data in DFs

In [ ]:
!pip install openpyxl

import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
import os
import gc
import gensim
import base64
import requests
import pandas as pd
import os
import nltk

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd 
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
!pip install transformers
import transformers
import nltk
nltk.download('wordnet')

In [ ]:
# install simpletransformers
!pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

#  French

In [ ]:
df_backtrans=pd.read_excel(r'../input/backtranslation-subtask1/FR_BT.xlsx',index_col=0)
df_backtrans

In [ ]:
df_backtrans.index = df_backtrans.index.astype(int)
df_backtrans[['Labels']] = df_backtrans[['Labels']].astype(int)

In [ ]:
df_backtrans

In [ ]:
df_backtrans=df_backtrans.sort_values(['ID'])

df_backtrans


# 1] Load pretrained model

In [ ]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "use_early_stopping":True,
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
        
    "electra","dbmdz/electra-base-french-europeana-cased-discriminator",
    args=train_args
)

# Train model Stage 1 NLP aug

In [ ]:
df_nlp_auf=pd.read_excel(r'../input/nlpaugfr/train_aug_subtask_fr.xlsx',index_col=0)
df_nlp_auf

In [ ]:
model.train_model(df_nlp_auf, learning_rate=3e-5)

#  test after stage 1

### Read Pablo test

In [ ]:
df_test = pd.read_csv('https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/test/official_test_set_with_labels/subtask-1/Fr-Subtask1-labels.tsv', sep='\t')

df_test=df_test.set_index('ID')
del df_test['Construction']

df_test

In [ ]:
df_test

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

result1, model_outputs1, wrong_predictions1 = model.eval_model(df_test, f1=f1_score, r=recall_score, p=precision_score)

result1

# 2] Train model stage 2 (bt+ original)

In [ ]:
df_backtrans

## French

In [ ]:
df_fr_train = df_backtrans[["Sentence","Labels"]]
df_fr_train

## english

In [ ]:
df_en_train = df_backtrans[["EN-FR","Labels"]]
df_en_train

In [ ]:
df_en_train=df_en_train.rename(columns={"EN-FR": "Sentence"})
df_en_train

## Italian

In [ ]:
df_it_train = df_backtrans[["IT-FR","Labels"]]
df_it_train

In [ ]:
df_it_train=df_it_train.rename(columns={"IT-FR": "Sentence"})
df_it_train

## Concat and remove duplicates

In [ ]:
df_all_train_bt=pd.concat([df_en_train,df_it_train,df_fr_train])
df_all_train_bt

In [ ]:
len(set(df_all_train_bt['Sentence']))

In [ ]:
df_all_train_bt=df_all_train_bt.drop_duplicates(subset='Sentence')

In [ ]:
df_all_train_bt["no_punctuation"] = df_all_train_bt['Sentence'].str.replace('[^\w\s]','')
df_all_train_bt

In [ ]:
df_all_train_bt=df_all_train_bt.drop_duplicates(subset='no_punctuation')
df_all_train_bt.sort_values('ID')

In [ ]:
del df_all_train_bt['no_punctuation']

In [ ]:
df_all_train_bt

# Train now

In [ ]:
# Train the model
model.train_model(df_all_train_bt)

# Test

In [ ]:
import base64
import requests

df_test 

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(df_test, f1=f1_score, r=recall_score, p=precision_score)


In [ ]:
result

# Save 

In [ ]:
y_dev_pred = [int(el[1] > el[0]) for el in model_outputs]
df_test['Predicted Labels']=y_dev_pred

In [ ]:
df_test

In [ ]:
#convert to TSV, i.e CSV with seperator= tab
results_tsv=df_test.to_csv('answer_with_sent_fr_subtask1_20_jan.tsv', sep="\t")

In [ ]:
del df_test['Sentence']


In [ ]:
#convert to TSV, i.e CSV with seperator= tab
results_tsv=df_test.to_csv('answer_fr_subtask1_20_jan.tsv', sep="\t")